# Learning Machine (Scratch)
##### Trying out ideas for classifiers bank

How sparse are the weights learned by LinearSVC with l1 penatly, let's try with a dummy dataset.
first 3 columns are noise, last two are actual data, and output is and function.

In [8]:
import numpy as np
from sklearn import svm

In [14]:
noise_columns = np.random.randn(9, 3)
data_columns = np.array( [ [1,0], [0,1], [1,1], [0,0]]*2 + [[0,1]])
X = np.hstack([noise_columns, data_columns]) # ttoal data
print X
y = np.array([0, 0,1,0]*2 + [0])
print y

[[ 1.4494588  -0.87701341  0.59572656  1.          0.        ]
 [-0.18027455 -0.20010864 -0.02353168  0.          1.        ]
 [-0.35538212 -0.01001067 -1.30983095  1.          1.        ]
 [ 0.86115974  0.41442813 -0.25017291  0.          0.        ]
 [-0.69212932  0.59595851 -0.22195596  1.          0.        ]
 [-0.23242998  0.84134878 -1.12742011  0.          1.        ]
 [ 0.22090338  0.05101463 -1.6615725   1.          1.        ]
 [ 0.68491212 -1.29508922 -0.46963583  0.          0.        ]
 [-0.25382081 -1.31338617 -2.7919427   0.          1.        ]]
[0 0 1 0 0 0 1 0 0]


In [15]:
clfl1 = svm.LinearSVC(dual=False,penalty='l1')
clfl2 = svm.SVC()

In [16]:
clfl1.fit(X,y)
clfl2.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [18]:
print clfl1.coef_

[[ 0.          0.02212106 -0.14678824  0.8860498   0.50087378]]


In [19]:
clfl2.get_params

<bound method SVC.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)>

In [25]:
clfl2.epsilon

0.0

In [26]:
cfll1_c10 = svm.LinearSVC(dual=False,penalty='l1',C=10).fit(X,y)
cfll1_c100 = svm.LinearSVC(dual=False,penalty='l1',C=100).fit(X,y)
cfll1_cp1 = svm.LinearSVC(dual=False,penalty='l1',C=0.1).fit(X,y)

In [30]:
print cfll1_c10.coef_, cfll1_c10.intercept_
print cfll1_c100.coef_, cfll1_c100.intercept_
print cfll1_cp1.coef_, cfll1_cp1.intercept_

[[ 0.          0.00223321 -0.01470972  1.88852809  1.84993938]] [-2.83541019]
[[  0.00000000e+00   2.02605890e-04  -1.44104685e-03   1.98892948e+00
    1.98513853e+00]] [-2.98367241]
[[ 0.  0.  0.  0.  0.]] [ 0.]


In [40]:
print cfll1_c10.score(X,y)
print cfll1_c100.score(X,y)
print cfll1_cp1.score(X,y)

1.0
1.0
0.777777777778


In [29]:
cfll1_c10.intercept_

array([-2.83541019])

Let us try dictionary learning, lassso etc


In [34]:
from sklearn import linear_model
clf_lasso = linear_model.Lasso(alpha=0.1)
clf_lasso.fit(X,y)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute='auto', tol=0.0001,
   warm_start=False)

In [35]:
clf_lasso.coef_

array([-0.        ,  0.        , -0.06317879,  0.11288796,  0.        ])

In [36]:
clf_lasso.predict(X)

array([ 0.19633389,  0.12256991,  0.31672471,  0.13688883,  0.24799408,
        0.19231225,  0.33894732,  0.15075423,  0.29747477])

In [38]:
clf_lasso.score(X,y)

0.237656379753451

#### Observations
Looks like LinearSVC gets score of 1, but lasso gets very low score. On the other hand, most of lasso's coeff are 0 unlike linearSVC.
Let us try dictionary sparse coder from ['http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparseCoder.html#sklearn.decomposition.SparseCoder']

In [44]:
from sklearn.decomposition import SparseCoder
clf_spc = SparseCoder(dictionary=X.T)
codes = clf_spc.transform(y)

In [45]:
codes

array([ 0.        ,  0.        , -2.97140344,  0.        ,  0.        ])

In [46]:
clf_spc

SparseCoder(dictionary=None, n_jobs=1, split_sign=False,
      transform_algorithm='omp', transform_alpha=None,
      transform_n_nonzero_coefs=None)

In [49]:
omp_coeff = linear_model.orthogonal_mp(X,y)
print omp_coeff

[ 0.          0.         -2.97140344  0.          0.        ]


This is same as above. In other words orhtogonal_mp is the same as SparseCoder because the default algo of sparse coder is 'omp'

In [53]:
dec_fx = clfl1.decision_function(X)
print dec_fx

[-0.57628523 -0.85558758  0.22348099 -1.30959889 -0.42367536 -0.6705116
  0.27646246 -1.31520064 -0.47384427]


In [54]:
from scipy.stats import logistic
logistic.pdf(dec_fx)

array([ 0.23034054,  0.2093018 ,  0.24690432,  0.16737478,  0.23910849,
        0.22387972,  0.24528323,  0.16683576,  0.23647588])

In [58]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

sigm_arr = [sigmoid(i) for i in dec_fx]
print sigm_arr

[0.359787808233521, 0.298262049836167, 0.5556388729819498, 0.21255397241002805, 0.39563760414019666, 0.3383822942416873, 0.5686787366650302, 0.2116178900361327, 0.3837067606497525]


In [62]:
X_big=np.vstack([X]*10)
print X_big.shape

(90L, 5L)


In [66]:
y_big = np.hstack([y]*10)
print y_big

[0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0
 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0]


In [67]:
clf_l1_big = svm.LinearSVC(dual=False,penalty='l1').fit(X_big,y_big)
print clf_l1_big.coef_

[[ 0.          0.00222808 -0.01469642  1.88854734  1.84997612]]


In [70]:
big_dec = clf_l1_big.decision_function(X)
print big_dec

[-0.95760743 -0.98556957  0.92230532 -2.83084563 -0.94230852 -0.96702591
  0.92761063 -2.83142926 -0.94736432]


In [72]:
sigm_arr = [sigmoid(i*10) for i in big_dec]
print sigm_arr

[6.93639249081493e-05, 5.244486518408945e-05, 0.9999012729818527, 5.079180141757308e-13, 8.082970547733213e-05, 6.312951111957026e-05, 0.9999063737637439, 5.049622937236995e-13, 7.684501014438589e-05]


In [74]:
clf_l1_big = svm.LinearSVC(dual=False,penalty='l1',C=100).fit(X_big,y_big)
print clf_l1_big.coef_

[[ 0.01220647  0.03976548 -0.05788358  2.03273072  1.95429305]]


In [77]:
omp_coeff = linear_model.orthogonal_mp(X_big,y_big)
print omp_coeff

[  0.           0.         -29.71403442   0.           0.        ]


In [81]:
# Finding the rightmost nonzero column
M = np.eye(5)
M[:,-2:] = np.zeros([5,2])
print M

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


In [93]:
[np.any(M[:,i]) for i in range(5)][::-1].index(True)

2